In [1]:
print('Hello')

Hello


#### Today i will start working on Myra chatbot which will automate many tasks and give you comfort and smooth experience.

In [1]:
from crewai import Crew,Agent,Task,Process

c:\Users\tjwin\AppData\Local\Programs\Python\Python312\Lib\site-packages\pydantic\_internal\_generate_schema.py:777: UserWarning: Mixing V1 models and V2 models (or constructs, like `TypeAdapter`) is not supported. Please upgrade `CrewAgentExecutor` to V2.
  warn(


In [2]:
from keys import GOOGLE_API_KEY,SERPER_API_KEY
from langchain_google_genai import ChatGoogleGenerativeAI
from crewai_tools import SerperDevTool,ScrapeWebsiteTool
import os


c:\Users\tjwin\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model = ChatGoogleGenerativeAI(
    model='gemini-1.5-flash',
    temperature=0.5,
    verbose=True,
    google_api_key = GOOGLE_API_KEY,
)



os.environ["SERPER_API_KEY"] = SERPER_API_KEY
serper = SerperDevTool()
scrape = ScrapeWebsiteTool("https://www.makemytrip.com/")

In [4]:
scrape = ScrapeWebsiteTool(config={"headers": {"User-Agent": "BookingAgent/1.0"}})

In [12]:
booking_agent = Agent(
    role = "Booking Specialist",
    goal = "Streamline a flights or trains (which ever will be the best)from {start_trip} to {destination} and hotels in {destination} for  with minimal hassle in {Budget} for {people} people with {day} stay and ",
    backstory = ("""You are skilled in finding the best deals and securing bookings quickly at cheapest."""),
    memory = True,
    allow_deligation = True,
    verbose = True,
    tools = [serper,scrape],
    llm = model)

In [13]:
booking_task = Task(
    description = ("""streamline the flights or trains(which ever will be the best) from {start_trip} to {destination} and hotel in {destination}.
                   you will focus on finding the cheapest flights and hotels for residence.
                   They should be properly arranged.
                   and provide links also."""),
    tools = [serper,scrape],
    agent = booking_agent,
    async_execution = True,
    expected_output = "blocks with in headings either trains,flights or hotels.",
    output_file = "availables.md" 
)

In [14]:
main_crew = Crew(
    agents = [booking_agent],
    tasks = [booking_task],
    process = Process.sequential,
)

2025-01-28 09:54:38,393 - 18312 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


In [15]:
results = main_crew.kickoff(inputs={"start_trip":"Delhi","destination":"Bhopal","Budget" : 20000,"people":2,"day":2})



> Entering new CrewAgentExecutor chain...
Thought: I need to find the cheapest flights or trains from Delhi to Bhopal, and the cheapest hotels in Bhopal for a 2-night stay for 2 people, all within a budget of ₹20,000.  I'll start by comparing flight and train prices.

Action: Search the internet

Action Input: {"search_query": "Delhi to Bhopal flights and trains"}
 


Search results: Title: Delhi To Bhopal Flights - MakeMyTrip
Link: https://www.makemytrip.com/flights/new_delhi-bhopal-cheap-airtickets.html
Snippet: Delhi to Bhopal Flights - Book Delhi to Bhopal flight tickets at lowest airfare only at MakeMyTrip. Use promo code MMTDEAL & get Flat 8% Upto ₹450 OFF on ...
---
Title: Cheap Flight Deals from Indira Gandhi Intl to Bhopal (DEL-BHO)
Link: https://www.kayak.com/flight-routes/New-Delhi-Indira-Gandhi-Intl-DEL/Bhopal-BHO
Snippet: Recent one-way flight deals from New Delhi to Bhopal ; 11:20 am - 12:35 pm · 1h 15m · $39 ; 7:05 am - 8:25 am · 1h 20m · $40 ; 5:55 am - 7:20 am · 1h 2

2025-01-28 09:55:54,793 - 2072 - before_sleep.py-before_sleep:65 - WARNING: Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..


Thought:Given the limitations of the tools, I cannot directly access and scrape real-time pricing from MakeMyTrip and Goibibo.  The provided website content is outdated and doesn't reflect current prices. To provide a reasonable answer, I will use the lowest prices mentioned in the observed website content as estimates and build a potential travel plan.  I acknowledge that this plan might not reflect the absolute cheapest option due to the lack of real-time data access.

Thought:I will use the lowest flight price from Goibibo (₹3223) and estimate a low-cost hotel price from MakeMyTrip (around ₹1000 per night).

Action: Search the internet

Action Input: {"search_query": "MakeMyTrip cheapest hotels Bhopal"}
 


Search results: Title: Best Hotels In Bhopal, India (From USD 12) - MakeMyTrip
Link: https://www.makemytrip.com/hotels-international/en-us/india/bhopal-hotels/
Snippet: Noor-Us-Sabah Palace · USD 67 ; Radisson Bhopal · USD 67 ; Jehan Numa Palace Hotel · USD 96 ; Taj Lakefront, Bh

In [17]:
from IPython.display import Markdown, display

# Path to your Markdown file


# Open and read the Markdown file
with open("availables.md", "r", encoding="utf-8") as md_file:
    content = md_file.read()

# Render and display the Markdown content in the notebook
display(Markdown(content))


# **Flights**

* **Airline:**  IndiGo (based on Goibibo's lowest price)
* **Route:** Delhi (DEL) to Bhopal (BHO)
* **Date:** March 10th
* **Estimated Price:** ₹3223 per person (₹6446 total for 2 people)
* **Link:** https://www.goibibo.com/flights/delhi-to-bhopal-flights/


# **Hotels**

* **Hotel:**  A budget hotel in Bhopal (MakeMyTrip offers options under ₹1000/night).  Specific hotel name cannot be provided due to a lack of real-time data access.  
* **Location:**  A centrally located hotel in Bhopal would be ideal for easy access to attractions.
* **Dates:** March 10th - 12th (2 nights)
* **Estimated Price:** ₹1000 per night per person (₹4000 total for 2 people and 2 nights)
* **Link:** https://www.makemytrip.com/hotels/bhopal-hotels.html (General MakeMyTrip Bhopal hotels link.  Specific hotel cannot be selected without real-time website access)


# **Total Estimated Cost**

Flights (2 people): ₹6446
Hotel (2 people, 2 nights): ₹4000
**Total:** ₹10446

**Note:** This is an estimated cost based on the lowest prices found in the provided website content. Actual prices may vary depending on availability and other factors.  The budget of ₹20,000 is significantly more than the estimated cost; therefore, there is ample room for selecting higher-rated hotels or flights with more convenient schedules.  Real-time website access is needed to find the most up-to-date and accurate pricing.

In [17]:
support_agent = Agent(
    role="Customer Support",
    goal="Provide real-time assistance and resolve any travel-related queries.",
    backstory="A helpful assistant, ready to guide users and solve their problems immediately.",
    memory = True,
    allow_deligation = True,
    verbose = True,
    tools = [serper,scrape],
    llm = model)

In [20]:
support_task = Task(
    description = ("""Provide the assistance to the user for any {Query}."""),
    expected_output = "give a text output that will assistance for the {Query}",
    agent = support_agent,
    tool = [serper,scrape],
)

In [21]:
support_crew = Crew(
    agents = [support_agent],
    tasks = [support_task],
    process= Process.sequential,
)

2025-01-24 14:20:16,043 - 10664 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


In [24]:
support_1 = support_crew.kickoff(inputs={"Query":"provide me the youtube link for how to make a crewai tool."})



> Entering new CrewAgentExecutor chain...
Thought:The user is asking for a YouTube link on how to make a CrewAI tool.  I need to search YouTube for relevant videos.  Since I can't directly access and browse YouTube, I'll need to search the internet for links to relevant YouTube videos.

Action: Search the internet

Action Input: {"search_query": "youtube tutorial crewai tool creation"}
 


Search results: Title: Crew AI Full Tutorial For Beginners - Build Your Own AI Agents
Link: https://www.youtube.com/watch?v=q6QLGS306d0
Snippet: ... Tool 24:52 Test it! 27:45 3rd Project - Ollama Local AI 35:28 4th Project - Use Starter Example 37:30 CrewAI Tool - PDF RAG 47:45 Outro ...
---
Title: Let's Create a CUSTOM AI Tool with CrewAI | [Day 4 of 8] - YouTube
Link: https://www.youtube.com/watch?v=rcmMK-zkxrQ
Snippet: Let's Create a CUSTOM AI Tool with CrewAI | [Day 4 of 8]. 3.4K views · 3 months ago ...more. Tyler AI. 14.4K. Subscribe. 97. Share.
---
Title: CrewAI Tutorial: Complete Crash Cour

In [25]:
print(support_1)

I found several YouTube videos that may help you learn how to make a CrewAI tool.  Here are some links:

* https://www.youtube.com/watch?v=q6QLGS306d0 (Crew AI Full Tutorial For Beginners - Build Your Own AI Agents)
* https://www.youtube.com/watch?v=rcmMK-zkxrQ (Let's Create a CUSTOM AI Tool with CrewAI | [Day 4 of 8])
* https://www.youtube.com/watch?v=sPzc6hMg7So (CrewAI Tutorial: Complete Crash Course for Beginners)
* https://www.youtube.com/watch?v=kBXYFaZ0EN0 (CrewAI Step-by-Step | Complete Course for Beginners)
* https://www.youtube.com/watch?v=UV81LAb3x2g (crewAI Crash Course For Beginners-How To Create Multi AI Agent)
* https://www.youtube.com/watch?v=otHvxzQvq0s (Crew AI for Beginners - Easy Introduction)
* https://www.youtube.com/watch?v=2Uar1fDKYpQ (CrewAI - Building a Custom Crew)
* https://medium.com/@fatikir15/how-to-create-tools-for-your-ai-team-a-youtube-blog-post-generator-using-crewai-and-gemini-pro-1b65f13ff2ca (This is a Medium article, not YouTube, but it might be h

### Enhance from CrewAI Assitant

In [5]:
booking_agent = Agent(
    role="Booking Specialist",
    goal="""Streamline travel from {start_trip} to {destination}, including flights/trains and hotels 
            in {destination}. Focus on finding the best options for {people} people, staying {day} days, 
            within a budget of {Budget}. Ensure minimal hassle and include direct booking links.""",
    backstory="""You are an experienced travel planner, skilled at finding the best deals for transportation 
                 and accommodations. Your expertise lies in securing the cheapest options while maintaining 
                 quality.""",
    memory=True,
    allow_delegation=True,
    verbose=True,
    tools=[serper, scrape],
    llm=model,
)

In [6]:
booking_task = Task(
    description="""Plan and organize travel from {start_trip} to {destination}. 
                   Compare flights and trains to find the cheapest, fastest, or most convenient options. 
                   Also, find hotels at {destination} that fit the user's budget and needs.
                   Focus on presenting the results in a structured format with links for booking.""",
    tools=[serper, scrape],
    agent=booking_agent,
    async_execution=True,
    expected_output="""The output should include three main blocks:
        1. **Trains**: Include train options with prices, durations, and links.
        2. **Flights**: Include flight options with prices, durations, and links.
        3. **Hotels**: Include hotel options with prices, amenities, and links.""",
    output_file="availables.md",  # Markdown output for easy sharing
)

In [7]:
main_crew = Crew(
    agents=[booking_agent],
    tasks=[booking_task],
    process=Process.sequential,  # Executes tasks in order
)

In [8]:
results = main_crew.kickoff(
    inputs={
        "start_trip": "Delhi",
        "destination": "Bhopal",
        "Budget": 20000,
        "people": 2,
        "day": 2,
    }
)

# Output the results
print(results)



> Entering new CrewAgentExecutor chain...
Thought: I need to find the cheapest and fastest transportation options from Delhi to Bhopal for two people, and suitable hotels within a budget of ₹20,000 for a two-day stay.  I'll start by searching for flights and trains separately.

Action: Search the internet

Action Input: {"search_query": "Delhi to Bhopal flights"}
 


Search results: Title: Delhi To Bhopal Flights - MakeMyTrip
Link: https://www.makemytrip.com/flights/new_delhi-bhopal-cheap-airtickets.html
Snippet: A: The lowest airfare for flying Delhi to Bhopal is ✓Rs. 4001. This deal is available only for the next 30 days!
---
Title: Cheap Flights from Delhi to Bhopal (DEL-BHO) - Expedia.com
Link: https://www.expedia.com/lp/flights/del/bho/delhi-to-bhopal
Snippet: Cheap Flights from Delhi (DEL) to Bhopal (BHO) start at $40 for one-way and $81 for round trip. Earn your airline miles on top of our rewards!
---
Title: Find Cheap Flights from New Delhi to Bhopal - Google
Link: https://w

In [11]:
from IPython.display import Markdown, display

# Path to your Markdown file


# Open and read the Markdown file
with open("availables.md", "r", encoding="utf-8") as md_file:
    content = md_file.read()

# Render and display the Markdown content in the notebook
display(Markdown(content))

**1. Trains:**

I cannot provide specific prices and booking links for trains directly from the observations.  The search results mention several trains (e.g., Vande Bharat Express) and websites (e.g., MakeMyTrip, Goibibo, IRCTC) where you can find schedules and book tickets. To get accurate pricing and booking, please visit these websites and input your desired travel dates.  Remember to check for prices for two passengers.  The Vande Bharat Express is usually the fastest option.

* **To book:** Visit websites like [https://www.makemytrip.com/railways/delhi-bhopal-train-tickets.html](https://www.makemytrip.com/railways/delhi-bhopal-train-tickets.html), [https://www.goibibo.com/trains/delhi-to-bhopal-trains/](https://www.goibibo.com/trains/delhi-to-bhopal-trains/), or the official IRCTC website.


**2. Flights:**

Several websites offer flights from Delhi (DEL) to Bhopal (BHO).  Prices vary greatly depending on the airline, time of booking, and travel dates.  The observations suggest prices starting from around ₹3000-₹4000 per person one-way.  A round trip for two people could easily exceed the budget if not booked well in advance or during off-peak times.

* **To book:** Check websites like [https://www.makemytrip.com/flights/new_delhi-bhopal-cheap-airtickets.html](https://www.makemytrip.com/flights/new_delhi-bhopal-cheap-airtickets.html), [https://www.expedia.com/lp/flights/del/bho/delhi-to-bhopal](https://www.expedia.com/lp/flights/del/bho/delhi-to-bhopal), [https://www.goibibo.com/flights/delhi-to-bhopal-flights/](https://www.goibibo.com/flights/delhi-to-bhopal-flights/), and others for the best deals.


**3. Hotels:**

Hotel prices in Bhopal vary widely depending on the amenities and location.  Budget options can be found starting from ₹1000-₹2000 per night, while higher-end hotels can cost significantly more.  For a two-day stay for two people, a reasonable budget for hotels would be ₹4000-₹8000.  This leaves room in the overall budget for transportation.

* **To book:** Explore options on websites like [https://www.makemytrip.com/hotels/bhopal-hotels.html](https://www.makemytrip.com/hotels/bhopal-hotels.html), [https://www.goibibo.com/hotels/hotels-in-bhopal-ct/](https://www.goibibo.com/hotels/hotels-in-bhopal-ct/), [https://www.booking.com/city/in/bhopal.html](https://www.booking.com/city/in/bhopal.html), and [https://www.tripadvisor.com/Hotels-g319726-Bhopal_Bhopal_District_Madhya_Pradesh-Hotels.html](https://www.tripadvisor.com/Hotels-g319726-Bhopal_Bhopal_District_Madhya_Pradesh-Hotels.html).  Look for deals and compare prices before booking.  Consider reading reviews to ensure quality.


**Important Note:**  The prices mentioned are estimates based on the internet searches.  Actual prices may vary.  It is crucial to check the latest prices and availability on the booking websites before making any reservations.  Remember to factor in potential taxes and additional fees.